# Tensorflow

- session defonition
- variables, constants, placeholders
- auto differentiation

Chap. 9 of A. Geron - Hands-On Machine Learning with Scikit-learn & Tensorflow 

In [1]:
# filter out future warnings from numpy on tensorflow calls
import warnings  
import numpy as np

with warnings.catch_warnings():  
    warnings.filterwarnings("ignore",category=FutureWarning)
    import tensorflow as tf

from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

## Dataset Loading  and scaling

In [2]:
housing = fetch_california_housing()
m, n = housing.data.shape
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m,1)), scaled_housing_data]

In [3]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.std(axis=0))

[ 1.00000000e+00  6.60969987e-17  5.50808322e-18  6.60969987e-17
 -1.06030602e-16 -1.10161664e-17  3.44255201e-18 -1.07958431e-15
 -8.52651283e-15]
[0. 1. 1. 1. 1. 1. 1. 1. 1.]


# Reduction to canonical form

\begin{align}
f\left(\theta\right) &= \frac{1}{m} \sum_{i=1}^{m} \left(\left[1, x^{(i)}\right] \theta - y^{(i)}\right)^2 = \frac{1}{m} \left(X\theta - y\right)^{T} \left(X\theta - y\right) = \\
&= \frac{1}{m} \theta^T X^T X \theta -2 \left(X^Ty\right)^T \theta + y^Ty = \\
&= \frac{2}{m} \left( \frac{1}{2} \theta^T H \theta - b^T \theta + \frac{c}{2} \right) = \\
g \left(\theta\right) &= \frac{2}{m} H \theta - b
\end{align}

With:
\begin{equation}
H = X^T X, \quad b = X^Ty, \quad c = y^Ty
\end{equation}

## Ridge Regression: direct solution
Consider also the possibility to introduce some regularization:
\begin{align}
f\left(\theta\right) &=
\frac{2}{m} 
\left(\frac{1}{2} \theta^T H \theta - b^T \theta + \frac{c}{2} + \frac{\lambda}{2} \theta^T\theta\right) = \\
g \left(\theta\right) &= \frac{2}{m} \left(H + \lambda I\right)\theta - b
\end{align}

With:
\begin{equation}
H = X^T X, \quad b = X^Ty, \quad c = y^Ty
\end{equation}

In [4]:
H = np.matmul(scaled_housing_data_plus_bias.T,scaled_housing_data_plus_bias)
k = np.linalg.cond(H)
print('Hessian condition number: {:4.2e}'.format(k))

Hessian condition number: 4.45e+01


In [5]:
class LinearRegressionDirect:
    def __init__ (self, X, y):
        self._X = X
        self._y = y
    
    def solve(self):
        self._define_model()
        with tf.Session(graph=self._graph) as sess:
            best_theta = tf.get_default_graph().get_tensor_by_name("theta:0").eval()
        
        return best_theta
    
    def _define_model(self):
        self._graph = tf.Graph()
        with self._graph.as_default():
            X = tf.constant(self._X, dtype=tf.float32, name="X")
            y = tf.constant(self._y, dtype=tf.float32, name="y")
            H = tf.matmul(tf.transpose(X), X)
            b = tf.matmul(tf.transpose(X), y)

            theta = tf.matmul(tf.matrix_inverse(H), b, name="theta")

In [6]:
lr = LinearRegressionDirect(X=scaled_housing_data_plus_bias, y=housing.target.reshape(-1, 1))
best_theta = lr.solve()

In [7]:
print(best_theta)
err = np.matmul(scaled_housing_data_plus_bias, best_theta) - housing.target.reshape(-1, 1)
print('mse: {:4.2e}'.format(np.sum(np.matmul(err.T,err))/m))

[[ 2.068557  ]
 [ 0.8296201 ]
 [ 0.11875182]
 [-0.26552784]
 [ 0.30569708]
 [-0.00450303]
 [-0.03932629]
 [-0.8998834 ]
 [-0.8705387 ]]
mse: 5.24e-01


## Gradient Descent

In [8]:
learning_rate = 1e-2

In [9]:
class GDError(Exception):
    """Base class for exceptions in this module."""
    pass

class GDNotFittedError(GDError):
    def __init__(self, message):
        self.message = message

class GDQuadratic_template:
    def __init__ (self, X, y, learning_rate=1e-2, n_epochs=2000, print_every=200):
        self._X = X
        self._y = y
        self._learning_rate = learning_rate
        self._n_epochs = n_epochs
        self._print_every = print_every
        self._best_theta = None
        
    def fit(self):
        self._define_model()
        with tf.Session(graph=self._graph) as sess:
    
            init = tf.get_default_graph().get_operation_by_name("init")
            training_op = tf.get_default_graph().get_operation_by_name("training_op")
            mse = tf.get_default_graph().get_tensor_by_name("mse:0")
            theta =  tf.get_default_graph().get_tensor_by_name("theta:0")
            
            sess.run(init)
    
            for epoch in range(self._n_epochs):
                if epoch % self._print_every == 0:
                    mse_k = mse.eval()
                    print('Epoch {:05d}, MSE={:4.2e}'.format(epoch,mse_k))
            
                sess.run(training_op)
    
            self._best_theta = theta.eval()
        
    def get_theta(self):
        if self._best_theta is None:
            raise GDNotFittedError()
            
        return self._best_theta
    
    def _define_model(self):
        pass

In [10]:
class GDQuadratic_1(GDQuadratic_template):
    def __init__ (self, X, y, learning_rate=1e-2, n_epochs=2000, print_every=200):
        super(GDQuadratic_1, self).__init__( X, y, learning_rate=learning_rate, n_epochs=n_epochs, print_every=print_every)
    
    def _define_model(self):
        self._graph = tf.Graph()
        with self._graph.as_default():
            X = tf.constant(self._X, dtype=tf.float32, name="X")
            y = tf.constant(self._y, dtype=tf.float32, name="y")
            H = tf.matmul(tf.transpose(X), X)
            b = tf.matmul(tf.transpose(X), y)
            
            # variables
            theta = tf.Variable(tf.zeros([n+1, 1]), name="theta")

            # initialization step
            init = tf.global_variables_initializer()

            # fval evaluation
            y_pred = tf.matmul(X, theta, name='predictions')        # X*theta
            error = y_pred - y
            mse = tf.reduce_mean(tf.square(error), name="mse")

            # gradient evaluation
            grad = 2/m * (tf.matmul(tf.transpose(H), theta) - b)

            # update step
            training_op = tf.assign(theta, theta - learning_rate * grad, name="training_op")

In [11]:
gd = GDQuadratic_1(X=scaled_housing_data_plus_bias, y=housing.target.reshape(-1, 1), learning_rate=learning_rate)
gd.fit()
best_theta = gd.get_theta()

Epoch 00000, MSE=5.61e+00
Epoch 00200, MSE=5.98e-01
Epoch 00400, MSE=5.63e-01
Epoch 00600, MSE=5.46e-01
Epoch 00800, MSE=5.36e-01
Epoch 01000, MSE=5.31e-01
Epoch 01200, MSE=5.28e-01
Epoch 01400, MSE=5.27e-01
Epoch 01600, MSE=5.26e-01
Epoch 01800, MSE=5.25e-01


In [12]:
print(best_theta)
err = np.matmul(scaled_housing_data_plus_bias, best_theta) - housing.target.reshape(-1, 1)
print('mse: {:4.2e}'.format(np.sum(np.matmul(err.T,err))/m))

[[ 2.0685511e+00]
 [ 8.4174490e-01]
 [ 1.2624303e-01]
 [-2.7867654e-01]
 [ 3.1227779e-01]
 [-1.9351706e-03]
 [-4.0218521e-02]
 [-8.3388931e-01]
 [-8.0548364e-01]]
mse: 5.25e-01


## SD with autodiff

In [13]:
learning_rate = 1e-2

In [14]:
class GDQuadratic_2(GDQuadratic_template):
    def __init__ (self, X, y, learning_rate=1e-2, n_epochs=2000, print_every=200):
        super(GDQuadratic_2, self).__init__( X, y, learning_rate=learning_rate, n_epochs=n_epochs, print_every=print_every)
    
    def _define_model(self):
        self._graph = tf.Graph()
        with self._graph.as_default():
            X = tf.constant(self._X, dtype=tf.float32, name="X")
            y = tf.constant(self._y, dtype=tf.float32, name="y")
            H = tf.matmul(tf.transpose(X), X)
            b = tf.matmul(tf.transpose(X), y)
            
            # variables
            theta = tf.Variable(tf.zeros([n+1, 1]), name="theta")

            # initialization step
            init = tf.global_variables_initializer()

            # fval evaluation
            y_pred = tf.matmul(X, theta, name='predictions')        # X*theta
            error = y_pred - y
            mse = tf.reduce_mean(tf.square(error), name="mse")

            # gradient evaluation
            grad = tf.gradients(mse, [theta])[0]

            # update step
            training_op = tf.assign(theta, theta - learning_rate * grad, name="training_op")

In [15]:
gd = GDQuadratic_2(X=scaled_housing_data_plus_bias, y=housing.target.reshape(-1, 1), learning_rate=learning_rate)
gd.fit()
best_theta = gd.get_theta()

Epoch 00000, MSE=5.61e+00
Epoch 00200, MSE=5.98e-01
Epoch 00400, MSE=5.63e-01
Epoch 00600, MSE=5.46e-01
Epoch 00800, MSE=5.36e-01
Epoch 01000, MSE=5.31e-01
Epoch 01200, MSE=5.28e-01
Epoch 01400, MSE=5.27e-01
Epoch 01600, MSE=5.26e-01
Epoch 01800, MSE=5.25e-01


In [16]:
print(best_theta)
err = np.matmul(scaled_housing_data_plus_bias, best_theta) - housing.target.reshape(-1, 1)
print('mse: {:4.2e}'.format(np.sum(np.matmul(err.T,err))/m))

[[ 2.0685525e+00]
 [ 8.4174454e-01]
 [ 1.2624320e-01]
 [-2.7867591e-01]
 [ 3.1227720e-01]
 [-1.9350675e-03]
 [-4.0218528e-02]
 [-8.3388835e-01]
 [-8.0548280e-01]]
mse: 5.25e-01


## Using baked solver

In [17]:
learning_rate = 1e-2

In [18]:
class GDQuadratic_3(GDQuadratic_template):
    def __init__ (self, X, y, learning_rate=1e-2, n_epochs=2000, print_every=200):
        super(GDQuadratic_3, self).__init__( X, y, learning_rate=learning_rate, n_epochs=n_epochs, print_every=print_every)
    
    def _define_model(self):
        self._graph = tf.Graph()
        with self._graph.as_default():
            X = tf.constant(self._X, dtype=tf.float32, name="X")
            y = tf.constant(self._y, dtype=tf.float32, name="y")
            H = tf.matmul(tf.transpose(X), X)
            b = tf.matmul(tf.transpose(X), y)
            
            # variables
            theta = tf.Variable(tf.zeros([n+1, 1]), name="theta")

            # initialization step
            init = tf.global_variables_initializer()

            # fval evaluation
            y_pred = tf.matmul(X, theta, name='predictions')        # X*theta
            error = y_pred - y
            mse = tf.reduce_mean(tf.square(error), name="mse")

            # define solver
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=self._learning_rate)

            # update step            
            training_op = optimizer.minimize(mse, name="training_op")

In [19]:
gd = GDQuadratic_3(X=scaled_housing_data_plus_bias, y=housing.target.reshape(-1, 1), learning_rate=learning_rate)
gd.fit()
best_theta = gd.get_theta()

Epoch 00000, MSE=5.61e+00
Epoch 00200, MSE=5.98e-01
Epoch 00400, MSE=5.63e-01
Epoch 00600, MSE=5.46e-01
Epoch 00800, MSE=5.36e-01
Epoch 01000, MSE=5.31e-01
Epoch 01200, MSE=5.28e-01
Epoch 01400, MSE=5.27e-01
Epoch 01600, MSE=5.26e-01
Epoch 01800, MSE=5.25e-01


In [20]:
print(best_theta)
err = np.matmul(scaled_housing_data_plus_bias, best_theta) - housing.target.reshape(-1, 1)
print('mse: {:4.2e}'.format(np.sum(np.matmul(err.T,err))/m))

[[ 2.0685525e+00]
 [ 8.4174454e-01]
 [ 1.2624320e-01]
 [-2.7867591e-01]
 [ 3.1227720e-01]
 [-1.9350675e-03]
 [-4.0218528e-02]
 [-8.3388835e-01]
 [-8.0548280e-01]]
mse: 5.25e-01


## Minibatch SSD

In [21]:
n_epochs = 2000
print_every = 200
learning_rate = 1e-2

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [22]:
class SGDQuadratic(GDQuadratic_template):
    def __init__ (self, X, y, learning_rate=1e-2, batch_size=200, n_epochs=2000, print_every=200):
        super(SGDQuadratic, self).__init__( X, y, learning_rate=learning_rate, n_epochs=n_epochs, print_every=print_every)
        self._X = X
        self._y = y
        self._learning_rate = learning_rate
        self._n_epochs = n_epochs
        self._print_every = print_every
        self._batch_size = min(batch_size, X.shape[0])
        self._n_batches = int(np.ceil(X.shape[0] / batch_size))
        self._best_theta = None
        
    def fit(self):
        self._define_model()
        with tf.Session(graph=self._graph) as sess:
            
            init = tf.get_default_graph().get_operation_by_name("init")
            training_op = tf.get_default_graph().get_operation_by_name("training_op")
            mse = tf.get_default_graph().get_tensor_by_name("mse:0")
            X = tf.get_default_graph().get_tensor_by_name("X:0")
            y = tf.get_default_graph().get_tensor_by_name("y:0")
            theta =  tf.get_default_graph().get_tensor_by_name("theta:0")
            
            sess.run(init)
            for epoch in range(self._n_epochs):
                for X_batch, y_batch in self._fetch_batch(epoch):
                    _, mse_k = sess.run([training_op, mse], feed_dict={X: X_batch, y: y_batch})

                if epoch % print_every == 0:
                    print('Epoch {:05d}, MSE={:4.2e}'.format(epoch,mse_k))

            self._best_theta = theta.eval()
    
    def _fetch_batch(self, epoch):
        for batch_index in range(self._n_batches):
            np.random.seed(epoch * self._n_batches + batch_index)
            indices = np.random.randint(self._X.shape[0], size=batch_size)
            X_batch = self._X[indices]
            y_batch = self._y[indices]    
            yield X_batch, y_batch
            
    def _define_model(self):
        self._graph = tf.Graph()
        with self._graph.as_default():
            X = tf.placeholder(tf.float32, shape=(None, self._X.shape[1]), name="X")
            y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
            H = tf.matmul(tf.transpose(X), X)                       
            b = tf.matmul(tf.transpose(X), y)
            
            # variables
            theta = tf.Variable(tf.zeros([n+1, 1]), name="theta")

            # initialization step
            init = tf.global_variables_initializer()

            # fval evaluation
            y_pred = tf.matmul(X, theta, name='predictions')
            error = y_pred - y
            mse = tf.reduce_mean(tf.square(error), name="mse")

            # define solver
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=self._learning_rate)

            # update step            
            training_op = optimizer.minimize(mse, name="training_op")
    

In [23]:
sgd = SGDQuadratic(X=scaled_housing_data_plus_bias, y=housing.target.reshape(-1, 1), learning_rate=learning_rate)
sgd.fit()
best_theta = sgd.get_theta()

Epoch 00000, MSE=8.02e-01
Epoch 00200, MSE=4.02e-01
Epoch 00400, MSE=5.92e-01
Epoch 00600, MSE=3.75e-01
Epoch 00800, MSE=1.99e+00
Epoch 01000, MSE=5.69e-01
Epoch 01200, MSE=4.62e-01
Epoch 01400, MSE=4.28e-01
Epoch 01600, MSE=7.02e-01
Epoch 01800, MSE=5.81e-01


In [24]:
print(best_theta)
err = np.matmul(scaled_housing_data_plus_bias, best_theta) - housing.target.reshape(-1, 1)
print('mse: {:4.2e}'.format(np.sum(np.matmul(err.T,err))/m))

[[ 2.07236   ]
 [ 0.835809  ]
 [ 0.12343293]
 [-0.25836352]
 [ 0.32831344]
 [-0.01244707]
 [-0.05894813]
 [-0.88290775]
 [-0.8470536 ]]
mse: 5.26e-01
